# 9th Day of Python 4 DIP

> Contents:
> 1. Derivation of images
> 2. Edges and Edge detection
 

Execute the cells to gain outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as SK
import time
from Tools import *
%matplotlib inline

### Derivation and Convolving them

#### The first calculation of Dx , Dy
$$ \Delta_x f = f(x,y) - f(x-1,y) $$
$$ \Delta_y f = f(x,y) - f(x,y-1) $$

In [ ]:
H_x = 0.5*np.array([[-1,1]])
H_y = H_x.copy().T
Camera = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)
Dx_Camera = Convolution2D(Camera,H_x)
Dy_Camera = Convolution2D(Camera,H_y)
ShowTimeMul((Camera,Dx_Camera,Dy_Camera),With_Hist=False,Im_title=["Original","Derivative x","Derivative y"],colorbar=False,cmaps=['gray']*3)

#### The first calculation of Dx , Dy
$$ \Delta_{2x} f = f(x+1,y) - f(x-1,y) $$
$$ \Delta_{2y} f = f(x,y+1) - f(x,y-1) $$

In [ ]:
H_x = 0.5*np.array([[-1,0,1]])
H_y = H_x.copy().T
Camera = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)
Dx_Camera = Convolution2D(Camera,H_x)
Dy_Camera = Convolution2D(Camera,H_y)
Magnitude_Camera = (Dx_Camera**2 + Dy_Camera**2)**0.5
Directions = np.arctan(Dy_Camera/Dx_Camera)
ShowTimeMul((Camera,Dx_Camera,Dy_Camera),With_Hist=False,Im_title=["Original","Derivative x","Derivative y"],colorbar=False,cmaps=['gray']*3)
ShowTimeMul((Magnitude_Camera,Directions),With_Hist=False,Im_title=["Magnitude","Directions"],colorbar=False,cmaps=['gray','hsv'])

### Using Scikit-image for derivative

In [ ]:
Sobel_h = SK.filters.sobel_h(Camera)
Sobel_v = SK.filters.sobel_v(Camera)
Prewitt_h = SK.filters.prewitt_h(Camera)
Prewitt_v = SK.filters.prewitt_v(Camera)
Schar_h = SK.filters.scharr_h(Camera)
Schar_v = SK.filters.scharr_v(Camera)
Farid_h = SK.filters.farid_h(Camera)
Farid_v = SK.filters.farid_v(Camera)
ShowTimeMul((Sobel_h,Sobel_v),With_Hist=False,Im_title=["Horizontal Sobel","Vertical Sobel"],colorbar=False,cmaps=['gray']*2)
ShowTimeMul((Prewitt_h,Prewitt_v),With_Hist=False,Im_title=["Horizontal Prewitt","Vertical Prewitt"],colorbar=False,cmaps=['gray']*2)
ShowTimeMul((Schar_h,Schar_v),With_Hist=False,Im_title=["Horizontal Scharr","Vertical Scharr"],colorbar=False,cmaps=['gray']*2)
ShowTimeMul((Farid_h,Farid_v),With_Hist=False,Im_title=["Horizontal Farid","Vertical Farid"],colorbar=False,cmaps=['gray']*2)


### Sharpening with Laplacian

$$\Delta_1 = \begin{bmatrix} 0 & -1 & 0\\-1 & 4 & -1\\0&-1&0\end{bmatrix}$$
$$\Delta_2 = \begin{bmatrix} -1 & -1 & -1\\-1 & 8 & -1\\-1&-1&-1\end{bmatrix}$$

In [ ]:
Laplacian1 = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
Laplacian2 = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
D2_Camera = FastConv2D(Camera,Laplacian1)
D2_Camera2 = FastConv2D(Camera,Laplacian2)
Sharpened1 = np.clip(Camera + D2_Camera,0,1)
Sharpened2 = np.clip(D2_Camera2 + Camera,0,1)
ShowTimeMul((Camera,Sharpened1,Sharpened2),With_Hist=False,Im_title=["Original","Sharped 1","Sharpened 2"],colorbar=False,cmaps=['gray']*3)


### Effect of Noise at Derivations

In [ ]:
Index = 5
F = Camera[Index,:]
DF = Sobel_v[Index,:]
Noise = SK.util.noise.random_noise(Camera,mode='s&p',seed=1000,salt_vs_pepper=0.2)
DNoise = SK.filters.sobel_v(Noise)[Index,:]

ShowTimeMul((Camera,Noise),With_Hist=False,Im_title=["Original","Noised"],colorbar=False,cmaps=['gray']*2)
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,5))
ax[0].plot(range(F.shape[0]),F,label="Signal")
ax[0].plot(range(F.shape[0]),DF,label="Dx")
ax[0].legend()

ax[1].plot(range(F.shape[0]),Noise[Index,:],label="Noised Signal")
ax[1].plot(range(F.shape[0]),DNoise,label="Dx Noised")
ax[1].legend()

### Edge detection with Scikit-Image

In [ ]:
X,Y = np.mgrid[-10:10:255j,-10:10:255j]
Image = np.cos(X**2+Y**2)
Dx_Image = -2*X*np.sin(X**2+Y**2)
Dy_Image = -2*Y*np.sin(X**2+Y**2)
angles = lambda x,y:  np.mod(np.arctan(y,x),np.pi)
Angles = angles(Dx_Image,Dy_Image)
ShowTimeMul((Image,Angles),With_Hist=False,Im_title=["Image","Angles"],colorbar=False,cmaps=['gray','hsv'])

In [ ]:
Sobel_Image = SK.filters.sobel(Image)
Scharr_Image = SK.filters.scharr(Image)
ShowTimeMul((Sobel_Image,Scharr_Image),With_Hist=False,Im_title=["Sobel","Scharr"],cmaps=['gray']*2,colorbar=False)

In [ ]:
Angles_Sobel = angles(SK.filters.sobel_h(Image),SK.filters.sobel_v(Image))
Angles_Scharr = angles(SK.filters.scharr_h(Image),SK.filters.scharr_v(Image))

ShowTimeMul((Angles_Sobel,Angles_Scharr),With_Hist=False,Im_title=["Angles Sobel","Angles Scharr"],colorbar=False,cmaps=['hsv']*2)

In [ ]:
Diff_Sobel = np.minimum(np.pi - np.abs(Angles - Angles_Sobel),np.abs(Angles - Angles_Sobel))
Diff_Scharr = np.minimum(np.pi - np.abs(Angles - Angles_Scharr),np.abs(Angles - Angles_Scharr))
ShowTimeMul((Diff_Sobel,Diff_Scharr),With_Hist=False,Im_title=["Diffs Sobel","Diffs Scharr"],figsize=(10,10),colorbar=True,cmaps=['inferno']*2)